In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import tensorflow as tf
from tensorflow import keras

In [ ]:
print("TF version: ", tf.__version__)


In [ ]:
if tf.__version__ < "2.0.0":
    tf.enable_eager_execution()
    print("Eager execution enabled.")
else:
    print("Eager execution enabled by default.")

In [ ]:
train = pd.read_csv('../input/sentiment-analysis-on-movie-reviews/train.tsv.zip', sep = '\t')
test = pd.read_csv('../input/sentiment-analysis-on-movie-reviews/test.tsv.zip', sep = '\t')

In [ ]:
print(train.shape)
print(test.shape)

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
train.info()

In [ ]:
test.info()

In [ ]:
import nltk
from tqdm import tqdm
from nltk.tokenize import word_tokenize 
from nltk.stem import WordNetLemmatizer
import re

In [ ]:
def sentence_cleaning(df):
    sentence = []
    for sent in tqdm(df['Phrase']):
        
        text = re.sub("[^a-zA-Z]"," ",sent)
        
        word = word_tokenize(text.lower())
        
        lemmatizer = WordNetLemmatizer()
        
        lemm_word = [lemmatizer.lemmatize(i) for i in word]
        
        sentence.append(lemm_word)
    return (sentence)


In [ ]:
train_sent = sentence_cleaning(train)
test_sent = sentence_cleaning(test)

In [ ]:
train_sent[0]

In [ ]:
train['Phrase'][0]

In [ ]:
print((' ').join(train_sent[0]))

In [ ]:
from keras.utils import to_categorical

In [ ]:
target_col = train.Sentiment.values
y_target = to_categorical(target_col)

In [ ]:
y_target.shape

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train,X_val,y_train,y_val = train_test_split(train_sent,y_target,test_size =0.2,stratify = y_target)

In [ ]:
X_train[11]

In [ ]:
print(len(X_train))
print(len(X_val))

In [ ]:
unique_words = set()
max_len = 0
for sent in tqdm(X_train):
    unique_words.update(sent)
    if(max_len < len(sent)):
        max_len = len(sent)
        sentence = sent

In [ ]:
print('Number of vocabs: ', len(list(unique_words)))
print('Max length of text is: ', max_len)

In [ ]:
sentence

In [ ]:
vocabulary = len(list(unique_words))
#max_len = max_len
oov = '<OOV>'
embedding_dim = 300
padding = 'post'
trunc = 'post'

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
tokenizer = Tokenizer(num_words = vocabulary,oov_token = oov,char_level = False)

In [ ]:
tokenizer.fit_on_texts(list(X_train))

In [ ]:
X_train = tokenizer.texts_to_sequences(X_train)
X_train = pad_sequences(X_train,maxlen = max_len,padding=padding,truncating = trunc)

In [ ]:
X_val = tokenizer.texts_to_sequences(X_val)
X_val = pad_sequences(X_val,maxlen = max_len,padding=padding,truncating = trunc)

In [ ]:
X_test = tokenizer.texts_to_sequences(test_sent)
X_test = pad_sequences(X_test,maxlen = max_len,padding=padding,truncating = trunc)

In [ ]:
print("X_training shape   : ",X_train.shape)
print("X_validation shape : ",X_val.shape)
print("X_testing shape    : ",X_test.shape)

In [ ]:
X_train[11]

In [ ]:
from keras.layers import Embedding,Bidirectional,Dense,LSTM,Dropout

In [ ]:
model = tf.keras.Sequential()

In [ ]:
model.add(Embedding(vocabulary,embedding_dim,input_length = max_len))
model.add(Bidirectional(LSTM(128, dropout = 0.8, recurrent_dropout=0.8, return_sequences=True)))
model.add(Bidirectional(LSTM(64,dropout = 0.5,recurrent_dropout=0.5,return_sequences=False)))
model.add(Dense(128,activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(5,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model.summary()

In [ ]:
num_epochs = 5

history = model.fit(X_train,
                    y_train,
                    validation_data = (X_val, y_val),
                    epochs = num_epochs,
                    batch_size = 256,
                    verbose = 1,
                    )

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend()
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.show()

In [ ]:
pred = np.argmax(model.predict(X_test),axis=-1)

In [ ]:
test_id = test['PhraseId']

In [ ]:
submission = pd.DataFrame({'PhraseID':test_id,'Sentiment':pred})
submission.to_csv('Submission.csv', index=False)
submission.head()